In [3]:
import tensorflow_addons as tfa
from tensorflow import keras

In [5]:
import numpy as np
encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

In [7]:
# STEP 2: Define basic parameters
vocab_size = 10000       # use actual vocab size
embed_size = 256         # dimension of embeddings
max_seq_len = 100        # max input sequence length (can be dynamic if masked)

embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)
encoder = keras.layers.LSTM(512, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

2025-07-07 10:40:15.868536: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [8]:
# STEP 4: Shared embedding layer
embedding_layer = keras.layers.Embedding(vocab_size, embed_size, name="shared_embedding")

encoder_embeddings = embedding_layer(encoder_inputs)  # (batch, time, embed)
decoder_embeddings = embedding_layer(decoder_inputs)


In [9]:
# STEP 5: Encoder — LSTM returns final hidden and cell state
encoder_lstm = keras.layers.LSTM(512, return_state=True, name="encoder_lstm")
_, state_h, state_c = encoder_lstm(encoder_embeddings)
encoder_state = [state_h, state_c]


In [12]:
# STEP 6: Define decoder with TrainingSampler
decoder_cell = keras.layers.LSTMCell(512, name="decoder_lstm_cell")
output_layer = keras.layers.Dense(vocab_size, name="output_projection")

sampler = tfa.seq2seq.sampler.TrainingSampler()

decoder = tfa.seq2seq.BasicDecoder(
    cell=decoder_cell,
    sampler=sampler,
    output_layer=output_layer,
    name="basic_decoder"
)

# decoder_outputs.rnn_output shape: (batch, time, vocab_size)
decoder_outputs, _, _ = decoder(
    decoder_embeddings,
    initial_state=encoder_state,
    sequence_length=sequence_lengths
)

Y_proba = tf.nn.softmax(decoder_outputs.rnn_output, axis=-1)


TypeError: Exception encountered when calling BasicDecoder.call().

[1mCould not automatically infer the output shape / dtype of 'basic_decoder' (of type BasicDecoder). Either the `BasicDecoder.call()` method is incorrect, or you need to implement the `BasicDecoder.compute_output_spec() / compute_output_shape()` method. Error encountered:

'NoneType' object is not subscriptable[0m

Arguments received by BasicDecoder.call():
  • args=('<KerasTensor shape=(None, None, 256), dtype=float32, sparse=False, ragged=False, name=keras_tensor_9>',)
  • kwargs={'initial_state': ['<KerasTensor shape=(None, 512), dtype=float32, sparse=False, ragged=False, name=keras_tensor_11>', '<KerasTensor shape=(None, 512), dtype=float32, sparse=False, ragged=False, name=keras_tensor_12>'], 'sequence_length': '<KerasTensor shape=(None,), dtype=int32, sparse=False, ragged=False, name=keras_tensor_2>'}

In [11]:
# STEP 7: Final model
model = keras.Model(
    inputs=[encoder_inputs, decoder_inputs, sequence_lengths],
    outputs=[Y_proba],
    name="Seq2Seq_Training_Model"
)
model.summary()


NameError: name 'Y_proba' is not defined